### Boiler X predictions

In [1]:
import pandas as pd 
import numpy as np 
from numpy import sort, sqrt, argsort, inf
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold

from sklearn.metrics import precision_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer, precision_recall_curve,confusion_matrix
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [29]:
import pickle

file_data = r'..\models\xgb_best_params_right_boiler_Y_feat_sel_thres_4.pkl'
file_data = open(file_data, 'rb')
xgb_best_params_edp_funct_model_boiler_y_sel_feat_th_4 = pickle.load(file_data)
file_data.close()

In [3]:
data_boiler_Y_year_3 = pd.read_csv('..\\data\\boiler_unity_xxx3.csv',sep=';')
data_boiler_Y_year_4 = pd.read_csv('..\\data\\boiler_unity_xxx4.csv',sep=';')

In [4]:
selected_colums_yirman = ['SSTIMESTAMP',
'Active Power',
'Boiler Feedwater Pressure',
'Boiler Furnace Pressure',
'Boiler Outlet Pressure',
'Coal Feeder #1',
'Coal Feeder #5',
'Dynamic Coal Classifier Rotational Speed Coal Mill #1',
'Dynamic Coal Classifier Rotational Speed Coal Mill #2',
'Dynamic Coal Classifier Rotational Speed Coal Mill #3',
'Secondary Air Row #3 (Side 2)',
'Secondary Air Row #5 (Side 2)',
'Boosted Overfire Air (Side 1)',
'Boosted Overfire Air (Side 2)',
'Secondary Air Fan #1 Temperature 2',
'Flue Gas Induced Draft Fan #1 Blade Pitch Angle',
'Total Atemperator Feedwater Flow',
'Main Steam First Desuperheater Control Valve position (Side 1)',
'Main Steam First Desuperheater Control Valve position (Side 2)',
'Main Steam Second Desuperheater Control Valve position (Side 1)',
'Main Steam Second Desuperheater Control Valve position (Side 2)',
'Main Steam Turbine Control Valve #B',
'Main Steam Turbine Control Valve #D',
'Air Heater #1 Differential Pressure',
'Reheated Steam Temperature 1 @Outlet']


data_Y_3 = data_boiler_Y_year_3[selected_colums_yirman]
data_Y_4 = data_boiler_Y_year_4[selected_colums_yirman]

In [5]:
datetime_values_3 = data_Y_3['SSTIMESTAMP']
hours_values_3 = pd.Series([x.split(' ')[1].split(':')[0] for x in datetime_values_3]) 
days_values_3 = pd.Series([x.split(' ')[0].split('-')[0] for x in datetime_values_3])
months_values_3 = pd.Series([x.split(' ')[0].split('-')[1] for x in datetime_values_3])

frequency_sampling_ID_3 = list([months_values_3[0]+'-'+days_values_3[0]+'-'+hours_values_3[0]])
for index in range(1, len(hours_values_3)):
    frequency_sampling_ID_3.append(months_values_3[index]+'-'+days_values_3[index]+'-'+hours_values_3[index])

data_Y_3['ID_sampling'] = frequency_sampling_ID_3 

datetime_values_4 = data_Y_4['SSTIMESTAMP']
hours_values_4 = pd.Series([x.split(' ')[1].split(':')[0] for x in datetime_values_4]) 
days_values_4 = pd.Series([x.split(' ')[0].split('-')[0] for x in datetime_values_4])
months_values_4 = pd.Series([x.split(' ')[0].split('-')[1] for x in datetime_values_4])

frequency_sampling_ID_4 = list([months_values_4[0]+'-'+days_values_4[0]+'-'+hours_values_4[0]])
for index in range(1, len(hours_values_4)):
    frequency_sampling_ID_4.append(months_values_4[index]+'-'+days_values_4[index]+'-'+hours_values_4[index])

data_Y_4['ID_sampling'] = frequency_sampling_ID_4


In [6]:
tablon_final_3 = pd.DataFrame({'ID_sampling': frequency_sampling_ID_3, 'HOUR': hours_values_3, 'DAY': days_values_3, 'MONTH': months_values_3})
tablon_final_3.drop_duplicates(subset='ID_sampling', keep='first', inplace=True)
tablon_final_3.set_index('ID_sampling', inplace=True)

features_3 = [x for x in data_Y_3.columns if x not in ['ID_sampling', 'HOUR', 'DAY', 'MONTH', 'SSTIMESTAMP']]

for col in features_3:
    #AÑADIR MEAN Y MEDIAN EXPONENTIALLY WEIGHTED
    data_hourly_grouped_3 = data_Y_3.groupby(['ID_sampling'])[col].agg(['median','mean','max','min','std','skew']).reset_index(drop=True)
    data_hourly_grouped_3 = data_hourly_grouped_3.add_prefix(col+'_')
    data_hourly_grouped_3.index = tablon_final_3.index
    tablon_final_3 = pd.concat([tablon_final_3, data_hourly_grouped_3],axis=1)


In [7]:
tablon_final_4 = pd.DataFrame({'ID_sampling': frequency_sampling_ID_4, 'HOUR': hours_values_4, 'DAY': days_values_4, 'MONTH': months_values_4})
tablon_final_4.drop_duplicates(subset='ID_sampling', keep='first', inplace=True)
tablon_final_4.set_index('ID_sampling', inplace=True)

features_4 = [x for x in data_Y_4.columns if x not in ['ID_sampling', 'HOUR', 'DAY', 'MONTH', 'SSTIMESTAMP']]

for col in features_4:
    #AÑADIR MEAN Y MEDIAN EXPONENTIALLY WEIGHTED
    data_hourly_grouped_4 = data_Y_4.groupby(['ID_sampling'])[col].agg(['median','mean','max','min','std','skew']).reset_index(drop=True)
    data_hourly_grouped_4 = data_hourly_grouped_4.add_prefix(col+'_')
    data_hourly_grouped_4.index = tablon_final_4.index
    tablon_final_4 = pd.concat([tablon_final_4, data_hourly_grouped_4],axis=1)


In [8]:
tablon_final_3.MONTH = tablon_final_3.MONTH.astype('int32')
tablon_final_3.DAY = tablon_final_3.DAY.astype('int32')
tablon_final_3.HOUR = tablon_final_3.HOUR.astype('int32')

tablon_final_3.sort_values(by=['MONTH', 'DAY', 'HOUR'], ascending=[1, 1, 1], inplace=True)

tablon_final_4.MONTH = tablon_final_4.MONTH.astype('int32')
tablon_final_4.DAY = tablon_final_4.DAY.astype('int32')
tablon_final_4.HOUR = tablon_final_4.HOUR.astype('int32')

tablon_final_4.sort_values(by=['MONTH', 'DAY', 'HOUR'], ascending=[1, 1, 1], inplace=True)


## Interpolamos

In [10]:
selected_important_features = ['Flue Gas Induced Draft Fan #1 Blade Pitch Angle_mean',
 'Main Steam Turbine Control Valve #B_max',
 'Secondary Air Row #5 (Side 2)_min',
 'Flue Gas Induced Draft Fan #1 Blade Pitch Angle_median',
 'Coal Feeder #1_mean',
 'Main Steam Second Desuperheater Control Valve position (Side 2)_median',
 'Boiler Outlet Pressure_max',
 'Boiler Furnace Pressure_mean',
 'Main Steam Turbine Control Valve #D_max',
 'Main Steam Second Desuperheater Control Valve position (Side 2)_mean',
 'Flue Gas Induced Draft Fan #1 Blade Pitch Angle_min',
 'Coal Feeder #1_min',
 'Secondary Air Fan #1 Temperature 2_min',
 'Air Heater #1 Differential Pressure_mean',
 'Boiler Outlet Pressure_median',
 'Coal Feeder #1_max',
 'Boosted Overfire Air (Side 1)_max',
 'Main Steam Turbine Control Valve #B_median',
 'Dynamic Coal Classifier Rotational Speed Coal Mill #1_min',
 'Main Steam Second Desuperheater Control Valve position (Side 2)_max',
 'Boiler Feedwater Pressure_max',
 'Boosted Overfire Air (Side 1)_min',
 'Secondary Air Fan #1 Temperature 2_mean',
 'Coal Feeder #1_median',
 'Flue Gas Induced Draft Fan #1 Blade Pitch Angle_max',
 'Boiler Feedwater Pressure_median',
 'Dynamic Coal Classifier Rotational Speed Coal Mill #2_min',
 'Dynamic Coal Classifier Rotational Speed Coal Mill #3_min',
 'Total Atemperator Feedwater Flow_max',
 'Main Steam First Desuperheater Control Valve position (Side 2)_min',
 'Boiler Outlet Pressure_mean',
 'Boiler Feedwater Pressure_std',
 'Active Power_mean',
 'Boosted Overfire Air (Side 1)_mean',
 'Boosted Overfire Air (Side 1)_median',
 'Coal Feeder #5_max']

In [11]:
inputs = [x for x in tablon_final_3.columns if x not in ['HOUR', 'DAY', 'MONTH']]

X_3 = tablon_final_3[selected_important_features]
X_4 = tablon_final_4[selected_important_features]

In [12]:
def imputeMissingInterpolating(array_df, features_to_impute):
    import numpy as np
    for attr in features_to_impute:
        attribute_interpolated = array_df[attr].interpolate(method='linear', limit_direction='both')
        #print('nans length for {}: {}'.format(attr, len(attribute_interpolated[np.isnan(attribute_interpolated)])))
        assert len(attribute_interpolated[np.isnan(attribute_interpolated)]) == 0
        
        array_df[attr] = attribute_interpolated
    
    return array_df

X_interpolated_3 = imputeMissingInterpolating(X_3, X_3.columns[X_3.isna().any()])
X_interpolated_4 = imputeMissingInterpolating(X_4, X_4.columns[X_4.isna().any()])


In [13]:
X_interpolated_4.head(3)

,Flue Gas Induced Draft Fan #1 Blade Pitch Angle_mean,Main Steam Turbine Control Valve #B_max,Secondary Air Row #5 (Side 2)_min,Flue Gas Induced Draft Fan #1 Blade Pitch Angle_median,Coal Feeder #1_mean,Main Steam Second Desuperheater Control Valve position (Side 2)_median,Boiler Outlet Pressure_max,Boiler Furnace Pressure_mean,Main Steam Turbine Control Valve #D_max,Main Steam Second Desuperheater Control Valve position (Side 2)_mean,...,Dynamic Coal Classifier Rotational Speed Coal Mill #2_min,Dynamic Coal Classifier Rotational Speed Coal Mill #3_min,Total Atemperator Feedwater Flow_max,Main Steam First Desuperheater Control Valve position (Side 2)_min,Boiler Outlet Pressure_mean,Boiler Feedwater Pressure_std,Active Power_mean,Boosted Overfire Air (Side 1)_mean,Boosted Overfire Air (Side 1)_median,Coal Feeder #5_max
ID_sampling,,,,,,,,,,,,,,,,,,,,,
01-01-00,55.649821,96.84207,68.03275,55.737300,24.865723,26.025390,-6.699219,-1.433919,56.85808,19.356282,...,114.6973,117.1387,40.05001,1.586914,-7.273763,0.425326,0.971101,65.214840,65.21484,25.06836
01-01-01,54.252116,96.54666,65.38943,54.174805,24.313966,19.482425,-6.718750,-1.463216,51.28557,20.245363,...,114.6973,117.1387,43.40686,12.475590,-7.210286,0.786905,0.956978,65.039060,65.03906,25.10742
01-01-02,53.542076,87.03445,65.87724,53.515630,24.139811,18.151855,-6.835938,-1.466472,28.48894,18.473309,...,114.6973,117.1387,46.93356,13.500980,-7.161458,0.509712,0.941863,65.009765,64.95117,24.52148


In [14]:
def scale_and_save_scaler(dataframe, features_to_scale, df_to_concat, saving_path=False):
    try:
        import pandas as pd
        from sklearn.preprocessing import StandardScaler
        from joblib import dump

        #husillo m22
        standard_scaler = StandardScaler()
        standard_scaler.fit(dataframe[features_to_scale].values)
        scaled_dataframe_values = standard_scaler.transform(dataframe[features_to_scale].values)
        scaled_dataframe = pd.DataFrame(columns=features_to_scale, data=scaled_dataframe_values, index = dataframe.index)
        
        features_no_to_scale = [x for x in dataframe.columns if x not in features_to_scale]
        
        assert dataframe.index.all() == scaled_dataframe.index.all()
        
        final_scaled_df = pd.concat([df_to_concat, scaled_dataframe], axis=1) 
        
        if saving_path:
            dump(standard_scaler, saving_path) 

        return final_scaled_df

    except Exception as exc:
        raise exc
        #return exc

#numeric_features = [x for x in tablon_final_3.columns if x not in ['HOUR', 'DAY', 'MONTH']]

dataset_to_concat_3 = tablon_final_3[['HOUR', 'DAY', 'MONTH']]
dataset_to_concat_4 = tablon_final_4[['HOUR', 'DAY', 'MONTH']]

X_scaled_interpolated_3 = scale_and_save_scaler(X_interpolated_3, features_to_scale = selected_important_features, df_to_concat = dataset_to_concat_3)
X_scaled_interpolated_4 = scale_and_save_scaler(X_interpolated_4, features_to_scale = selected_important_features, df_to_concat = dataset_to_concat_4)

In [15]:
X_scaled_interpolated_3.to_csv('../data/boiler_Y_year_3_train_data_scaled_interpolated_imp_feat.csv')
X_scaled_interpolated_4.to_csv('../data/boiler_Y_year_4_train_data_scaled_interpolated_imp_feat.csv')


In [23]:
X_scaled_interpolated_from_csv_3 = pd.read_csv('../data/boiler_Y_year_3_train_data_scaled_interpolated_imp_feat.csv')
X_scaled_interpolated_from_csv_3.set_index('ID_sampling', inplace=True)
X_scaled_interpolated_from_csv_4 = pd.read_csv('../data/boiler_Y_year_4_train_data_scaled_interpolated_imp_feat.csv')
X_scaled_interpolated_from_csv_4.set_index('ID_sampling', inplace=True)


In [24]:
X_scaled_interpolated_from_csv_3.head(3)

,HOUR,DAY,MONTH,Flue Gas Induced Draft Fan #1 Blade Pitch Angle_mean,Main Steam Turbine Control Valve #B_max,Secondary Air Row #5 (Side 2)_min,Flue Gas Induced Draft Fan #1 Blade Pitch Angle_median,Coal Feeder #1_mean,Main Steam Second Desuperheater Control Valve position (Side 2)_median,Boiler Outlet Pressure_max,...,Dynamic Coal Classifier Rotational Speed Coal Mill #2_min,Dynamic Coal Classifier Rotational Speed Coal Mill #3_min,Total Atemperator Feedwater Flow_max,Main Steam First Desuperheater Control Valve position (Side 2)_min,Boiler Outlet Pressure_mean,Boiler Feedwater Pressure_std,Active Power_mean,Boosted Overfire Air (Side 1)_mean,Boosted Overfire Air (Side 1)_median,Coal Feeder #5_max
ID_sampling,,,,,,,,,,,,,,,,,,,,,
01-01-00,0,1,1,-2.332397,-1.521259,-1.255896,-2.308661,-1.50666,-0.840168,1.901044,...,-2.110952,-2.35249,-1.189515,-1.12488,1.973156,-0.289632,-1.608938,-0.487889,-0.487419,-1.400309
01-01-01,1,1,1,-2.332397,-1.521259,-1.297074,-2.308661,-1.50666,-0.840168,1.912681,...,-2.110952,-2.35249,-1.189515,-1.12488,1.978802,-0.289632,-1.608860,-0.487889,-0.487419,-1.400309
01-01-02,2,1,1,-2.332397,-1.521259,-1.255358,-2.308661,-1.50666,-0.836632,1.959229,...,-2.110952,-2.35249,-1.189515,-1.12488,2.018326,-0.289632,-1.608781,-0.487889,-0.487419,-1.400309


### Pasamos a predecir:

In [25]:
numeric_features = [x for x in X_scaled_interpolated_from_csv_3.columns if x not in ['HOUR', 'DAY', 'MONTH']]

In [27]:
X_scaled_interpolated_from_csv_3[numeric_features]

,Flue Gas Induced Draft Fan #1 Blade Pitch Angle_mean,Main Steam Turbine Control Valve #B_max,Secondary Air Row #5 (Side 2)_min,Flue Gas Induced Draft Fan #1 Blade Pitch Angle_median,Coal Feeder #1_mean,Main Steam Second Desuperheater Control Valve position (Side 2)_median,Boiler Outlet Pressure_max,Boiler Furnace Pressure_mean,Main Steam Turbine Control Valve #D_max,Main Steam Second Desuperheater Control Valve position (Side 2)_mean,...,Dynamic Coal Classifier Rotational Speed Coal Mill #2_min,Dynamic Coal Classifier Rotational Speed Coal Mill #3_min,Total Atemperator Feedwater Flow_max,Main Steam First Desuperheater Control Valve position (Side 2)_min,Boiler Outlet Pressure_mean,Boiler Feedwater Pressure_std,Active Power_mean,Boosted Overfire Air (Side 1)_mean,Boosted Overfire Air (Side 1)_median,Coal Feeder #5_max
ID_sampling,,,,,,,,,,,,,,,,,,,,,
01-01-00,-2.332397,-1.521259,-1.255896,-2.308661,-1.506660,-0.840168,1.901044,2.384710,-1.584077,-0.878519,...,-2.110952,-2.352490,-1.189515,-1.124880,1.973156,-0.289632,-1.608938,-0.487889,-0.487419,-1.400309
01-01-01,-2.332397,-1.521259,-1.297074,-2.308661,-1.506660,-0.840168,1.912681,2.384710,-1.584077,-0.877913,...,-2.110952,-2.352490,-1.189515,-1.124880,1.978802,-0.289632,-1.608860,-0.487889,-0.487419,-1.400309
01-01-02,-2.332397,-1.521259,-1.255358,-2.308661,-1.506660,-0.836632,1.959229,2.384710,-1.584077,-0.874882,...,-2.110952,-2.352490,-1.189515,-1.124880,2.018326,-0.289632,-1.608781,-0.487889,-0.487419,-1.400309
01-01-03,-2.332397,-1.527003,-1.255205,-2.308661,-1.506004,-0.836632,1.959229,2.378986,-1.584077,-0.874882,...,-2.110952,-2.352490,-1.174771,-1.124880,2.029619,-0.289632,-1.608702,-0.487889,-0.487419,-1.400309
01-01-04,-2.332397,-1.527003,-1.255057,-2.308661,-1.505785,-0.836632,1.959229,2.350367,-1.584077,-0.874882,...,-2.110952,-2.352490,-1.174771,-1.127704,1.988213,-0.289632,-1.608637,-0.487889,-0.487419,-1.399438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12-31-19,0.106077,0.116115,-0.841130,0.110315,1.115148,0.605131,-0.356541,-0.567336,-0.006966,0.573825,...,0.328919,0.226345,0.586702,-0.870700,-0.282056,-0.213323,0.637506,-0.487889,-0.487419,-1.400309
12-31-20,0.099282,0.100956,-0.840740,0.095775,1.104495,0.124249,-0.350723,-0.532993,-0.049930,0.137137,...,0.328919,0.226345,0.920036,-0.430120,-0.300406,-0.174797,0.639302,-0.487889,-0.487419,-1.400309
12-31-21,0.088501,0.087481,-0.841096,0.095237,1.098439,-0.116192,-0.391452,-0.518684,-0.097190,-0.134846,...,0.328919,0.226345,1.207682,0.061295,-0.312640,-0.189912,0.639275,-0.487889,-0.487419,-1.400309


In [30]:
preds_probas_3_df = pd.DataFrame(data={'y_predicted_values': pd.Series(), 'y_predicted_probas': pd.Series()}, index=X_scaled_interpolated_from_csv_3.index)
boiler_Y_year_3_probs = xgb_best_params_edp_funct_model_boiler_y_sel_feat_th_4.predict_proba(X_scaled_interpolated_from_csv_3[selected_important_features])[:, 1]
boiler_Y_year_3_preds = xgb_best_params_edp_funct_model_boiler_y_sel_feat_th_4.predict(X_scaled_interpolated_from_csv_3[selected_important_features])

preds_probas_4_df = pd.DataFrame(data={'y_predicted_values': pd.Series(), 'y_predicted_probas': pd.Series()}, index=X_scaled_interpolated_from_csv_4.index)
boiler_Y_year_4_probs = xgb_best_params_edp_funct_model_boiler_y_sel_feat_th_4.predict_proba(X_scaled_interpolated_from_csv_4[numeric_features])[:, 1]
boiler_Y_year_4_preds = xgb_best_params_edp_funct_model_boiler_y_sel_feat_th_4.predict(X_scaled_interpolated_from_csv_4[numeric_features])


In [31]:
preds_probas_3_df.y_predicted_values = boiler_Y_year_3_preds
preds_probas_3_df.y_predicted_probas = boiler_Y_year_3_probs

positive_preds_probas_3_df = preds_probas_3_df[preds_probas_3_df.y_predicted_values==1]
positive_preds_probas_3_df

,y_predicted_values,y_predicted_probas
ID_sampling,,
01-11-10,1,0.638112
01-11-11,1,0.669647
01-11-12,1,0.669647
01-11-13,1,0.669647
01-11-14,1,0.669647
...,...,...
10-30-18,1,0.503400
11-03-07,1,0.508475
11-03-08,1,0.508475


In [32]:
unique_ID_values = positive_preds_probas_3_df.index
unique_days = [x.split('-')[1] for x in unique_ID_values]
unique_months = [x.split('-')[0] for x in unique_ID_values]

positive_preds_probas_3_df['month'] = unique_months
positive_preds_probas_3_df['day'] = unique_days

daily_median_pred_3_probs = positive_preds_probas_3_df.groupby(['month', 'day'])['y_predicted_probas'].agg(['median']) #.reset_index(drop=True)
daily_median_pred_3_probs_df = pd.DataFrame(daily_median_pred_3_probs)


In [33]:
daily_median_pred_3_probs_df.sort_values(by=['month', 'day'], axis='index')

median
month day          
01    11   0.669647
      20   0.707980
03    07   0.585349
      27   0.669647
      28   0.669647
04    03   0.653880
      07   0.669647
      08   0.669647
      15   0.587265
      16   0.725555
      17   0.725555
      21   0.725555
      22   0.725555
      23   0.638112
      24   0.638112
      26   0.725555
05    23   0.669647
      24   0.638112
      25   0.669647
      26   0.669647
06    21   0.725555
      22   0.725555
      28   0.725555
07    16   0.598847
      17   0.537471
      19   0.573670
      20   0.591916
      21   0.585281
      22   0.585281
08    12   0.560183
      13   0.560183
      14   0.560183
      16   0.585281
10    30   0.543846
11    03   0.508475

In [34]:
preds_probas_4_df.y_predicted_values = boiler_Y_year_4_preds
preds_probas_4_df.y_predicted_probas = boiler_Y_year_4_probs

positive_preds_probas_4_df = preds_probas_4_df[preds_probas_4_df.y_predicted_values==1]
positive_preds_probas_4_df

,y_predicted_values,y_predicted_probas
ID_sampling,,
01-05-08,1,0.564514
01-05-09,1,0.564514
01-05-10,1,0.504376
01-05-11,1,0.539148
01-05-12,1,0.539148
...,...,...
12-08-13,1,0.691934
12-08-14,1,0.691934
12-08-15,1,0.691934


In [35]:
unique_ID_values = positive_preds_probas_4_df.index
unique_days = [x.split('-')[1] for x in unique_ID_values]
unique_months = [x.split('-')[0] for x in unique_ID_values]

positive_preds_probas_4_df['month'] = unique_months
positive_preds_probas_4_df['day'] = unique_days

daily_median_pred_4_probs = positive_preds_probas_4_df.groupby(['month', 'day'])['y_predicted_probas'].agg(['median']) #.reset_index(drop=True)
daily_median_pred_4_probs_df = pd.DataFrame(daily_median_pred_4_probs)


In [36]:
daily_median_pred_4_probs_df.sort_values(by=['month', 'day'], axis='index')

median
month day          
01    05   0.539148
      06   0.539148
      13   0.704947
      14   0.704947
07    09   0.537471
11    26   0.653880
      27   0.669647
      28   0.669647
      29   0.669647
      30   0.669647
12    01   0.669647
      02   0.669647
      03   0.669647
      04   0.669647
      05   0.669647
      06   0.669647
      07   0.669647
      08   0.691934